In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")


In [2]:
## Data Ingestion
from langchain_community.document_loaders import WebBaseLoader
url="https://python.langchain.com/docs/integrations/providers/openai/"
loader=WebBaseLoader(url)
docs=loader.load()
docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://python.langchain.com/docs/integrations/providers/openai/', 'title': 'OpenAI | 🦜️🔗 LangChain', 'description': 'All functionality related to OpenAI', 'language': 'en'}, page_content='\n\n\n\n\nOpenAI | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchProvidersAnthropicAWSGoogleHugging FaceMicrosoftOpenAIMoreProvidersAbsoAcreomActiveloop Deep LakeAerospikeAI21 LabsAimAINetworkAirbyteAirtableAlchemyAleph AlphaAlibaba CloudAnalyticDBAnnoyAnthropicAnyscaleApache Software FoundationApache DorisApifyAppleArangoDBArceeArcGISArgillaArizeArthurArxivAscendAskNewsAssemblyAIAstra DBAtlasAwaDBAWSAZLyricsBAAIBagelBagelDBBaichuanBaiduBananaBasetenBeamBeautiful SoupBibTeXBiliBiliBittensorBlackboardbookend.aiBoxBrave SearchBreebs (Open Knowledge)Bro

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'https://python.langchain.com/docs/integrations/providers/openai/', 'title': 'OpenAI | 🦜️🔗 LangChain', 'description': 'All functionality related to OpenAI', 'language': 'en'}, page_content='OpenAI | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/integrations/providers/openai/', 'title': 'OpenAI | 🦜️🔗 LangChain', 'description': 'All functionality related to OpenAI', 'language': 'en'}, page_content='Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchProvidersAnthropicAWSGoogleHugging FaceMicrosoftOpenAIMoreProvidersAbsoAcreomActiveloop Deep LakeAerospikeAI21 LabsAimAINetworkAirbyteAirtableAlchemyAleph AlphaAlibaba CloudAnalyticDBAnnoyAnthropicAnyscaleApache Software FoundationApache DorisApifyAppleArangoDBArceeArcGISArgillaArizeArthurA

In [4]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()


In [5]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)


In [6]:
vectorstoredb

In [7]:
query="OpenAI is American artificial intelligence"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'OpenAI is American artificial intelligence (AI) research laboratory\nconsisting of the non-profit OpenAI Incorporated\nand its for-profit subsidiary corporation OpenAI Limited Partnership.\nOpenAI conducts AI research with the declared intention of promoting and developing a friendly AI.\nOpenAI systems run on an Azure-based supercomputing platform from Microsoft.\n\n\nThe OpenAI API is powered by a diverse set of models with different capabilities and price points.\nChatGPT is the Artificial Intelligence (AI) chatbot developed by OpenAI.'

In [8]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [15]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:
<context>
{context}
</context>
"""
)


In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)


In [18]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"OpenAI is American artificial intelligence",
    "context":[Document(page_content="OpenAI is American artificial intelligence (AI) research laboratory consisting of the non-profit OpenAI Incorporated and its for-profit subsidiary corporation OpenAI Limited Partnership. OpenAI conducts AI research with the declared intention of promoting and developing a friendly AI. OpenAI systems run on an Azure-based supercomputing platform from Microsoft.")]
})

'OpenAI is an American artificial intelligence research laboratory that operates as a combination of a non-profit, OpenAI Incorporated, and a for-profit subsidiary, OpenAI Limited Partnership. They conduct AI research with the aim of promoting and developing friendly AI, and their systems are run on an Azure-based supercomputing platform provided by Microsoft.'